In [1]:
import numpy as np
import pandas as pd
import sys, codecs
import dask.dataframe as dd
import dask.array as da
import dask.bag as db
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.cluster import KMeans
import math
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from gensim.models.phrases import Phrases,Phraser
from gensim.models import Word2Vec
from nltk.stem import WordNetLemmatizer
import ast
import os    
import glob
import datetime
from textblob import TextBlob
# load model well trained
frozen_model = Phraser.load('phrase_6g.pkl')
ngram_model=Word2Vec.load('ngram_onten.model')

In [35]:
df_b=pd.read_csv('UBDC/sale_2020/B/sale_valid_property_id_expanded_year_2020_area_B_part_00000.csv')
df_sim=pd.read_csv('UBDC/sale_2020/B/00000_token_sim.csv')


In [18]:
df_b['sen']=0
for count in range(df_b.shape[0]):
    blob = TextBlob(df_b.description[count])
    df_b['sen'].iloc[count]= blob.sentiment[0]

/opt/conda/envs/sds2021/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [5]:
TextBlob('spacious').sentiment

Sentiment(polarity=0.0, subjectivity=0.0)

In [41]:

TextBlob(df_sim['0'][7]).sentiment

Sentiment(polarity=0.16349206349206347, subjectivity=0.35128205128205126)

In [42]:
TextBlob(df_b['tfidf_key'][7]).sentiment

Sentiment(polarity=-0.1, subjectivity=0.18333333333333332)

In [44]:
# sentiment analysis
citys=['W','WC','E','EC','N','NW','SE','SW','B','L','M','G','BS','S']
# citys=['M']
for city in citys:
#     top500=get_top500(city)
    path=r'UBDC/sale_2020/'+city+'/'
    filepath=path+'*complex*.csv'
    for csvname in glob.glob(filepath):
        print(csvname)
#         get_keyword_input(csvname[0:-23]+'.csv',csvname,path)
#         get_dum_input(csvname[0:-23]+'.csv',top500,path)
        df=pd.read_csv(csvname[0:-23]+'.csv')
        df['polarity']=0
        df['subjectivity']=0
        for count in range(df.shape[0]):
            blob = TextBlob(df.description[count])
            df['polarity'].iloc[count]= blob.sentiment[0]
            df['subjectivity'].iloc[count]= blob.sentiment[1]
        df.to_csv(csvname[0:-23]+'.csv',index=False,encoding="utf_8_sig")

UBDC/sale_2020/W/sale_valid_property_id_expanded_year_2020_area_W_part_00005clean_token_complex.csv


/opt/conda/envs/sds2021/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


UBDC/sale_2020/W/sale_valid_property_id_expanded_year_2020_area_W_part_00002clean_token_complex.csv
UBDC/sale_2020/W/sale_valid_property_id_expanded_year_2020_area_W_part_00001clean_token_complex.csv
UBDC/sale_2020/W/sale_valid_property_id_expanded_year_2020_area_W_part_00000clean_token_complex.csv
UBDC/sale_2020/W/sale_valid_property_id_expanded_year_2020_area_W_part_00006clean_token_complex.csv
UBDC/sale_2020/W/sale_valid_property_id_expanded_year_2020_area_W_part_00003clean_token_complex.csv
UBDC/sale_2020/W/sale_valid_property_id_expanded_year_2020_area_W_part_00004clean_token_complex.csv
UBDC/sale_2020/WC/sale_valid_property_id_expanded_year_2020_area_WC_part_00000clean_token_complex.csv
UBDC/sale_2020/E/sale_valid_property_id_expanded_year_2020_area_E_part_00002clean_token_complex.csv
UBDC/sale_2020/E/sale_valid_property_id_expanded_year_2020_area_E_part_00001clean_token_complex.csv
UBDC/sale_2020/E/sale_valid_property_id_expanded_year_2020_area_E_part_00004clean_token_complex.cs

In [45]:
df_all_oneten=pd.read_csv('2020_token_oneten.csv')

In [48]:
df_all_oneten

,des_clea
0,"['onward', 'chain', 'charles', 'harding', 'ple..."
1,"['virtual', 'viewing', 'enquire', 'stunning', ..."
2,"['spacious', 'bedroom', 'detach', 'property', ..."
3,"['approached', 'driveway', 'property', 'provid..."
4,"['chestnut', 'house', 'stamp', 'pay', 'scheme'..."
...,...
473487,"['student', 'share', 'accommodation', 'spaciou..."
473488,"['modernisation', 'offer', 'upward', 'chain', ..."
473489,"['georgian', 'inspire', 'facade', 'feature', '..."
473490,"['bedroom', 'detach', 'family', 'offer', 'sale..."


In [50]:
%%time
vectorizer = CountVectorizer()  
X = vectorizer.fit_transform(df_all_oneten.des_clea)

CPU times: user 41.7 s, sys: 8.13 s, total: 49.8 s
Wall time: 49.8 s


In [51]:
Xcsc=X.tocsc()
SUM=pd.DataFrame(list(vectorizer.get_feature_names()),columns=['word'])
SUM['sum']=0
for count in range(len(vectorizer.get_feature_names())):
    SUM['sum'].iloc[count]=Xcsc.getcol(count).toarray().sum()

/opt/conda/envs/sds2021/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [53]:
SUM

,word,sum
0,___________________________________,2
1,__________________________________________,1
2,______________________________________________...,2
3,______________________________________________...,1
4,______________________________________________...,1
...,...,...
305774,ﬂood,1
305775,ﬂooded,2
305776,ﬂoor,18
305777,ﬂooring,5


In [56]:
df_token=pd.read_csv('UBDC/sale_2020/B/sale_valid_property_id_expanded_year_2020_area_B_part_00000clean_token_complex.csv')  #token file
#                 df=pd.read_csv(csvname[0:-23]+'.csv')  #
df_token.des_clea=df_token.des_clea.apply(ast.literal_eval)
for row in df_token.des_clea.tolist():
    print (frozen_model[row])

['natural', 'light', 'flow', 'offer', 'wealth', 'feature', 'characteristic', 'epitomes', 'design', 'property', 'property', 'comprise', 'porch', 'entrance', 'grand', 'entrance', 'hall', 'spacious', 'live', 'feature', 'bay', 'window', 'charming', 'fireplace', 'hallway', 'lead', 'spacious', 'dining', 'study', 'attached', 'dining', 'kitchen', 'afford', 'range', 'quality', 'unit', 'breakfast', 'bar', 'appliance', 'gas', 'hob', 'oven', 'lead', 'utility', 'match', 'unit', 'bathroom', 'upstairs', 'spaciously', 'sized', 'bedroom', 'luxurious', 'modern', 'family', 'bathroom', 'attach', 'study', 'property', 'boast', 'spacious', 'loft', 'conversion', 'suppling', 'property', 'extra', 'fitted', 'ceiling', 'window', 'natural', 'light', 'enclose', 'garden', 'patio', 'offer', 'ample', 'seating', 'bar', 'pond', 'grass', 'add', 'green', 'garden', 'property', 'basement', 'sidewalk', 'garden', 'double', 'storage', 'spacious', 'garage', 'floor', 'pit', 'driveway', 'fit', 'multiple', 'car', 'walking', 'dista

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



['innovate_estate', 'agents', 'pleased', 'bedroom', 'semi', 'detach', 'house', 'modernisation', 'situate', 'rowley_regis', 'property', 'boast', 'front', 'driveway', 'car', 'port', 'garage', 'lounge', 'guest', 'kitchen', 'bathroom', 'separate', 'rear', 'garden', 'double', 'glazing', 'gas', 'central', 'heating', 'spec', 'seek', 'location', 'property', 'stone_throw', 'range', 'day', 'day', 'amenity', 'educational', 'facility', 'transport', 'link', 'causeway', 'green', 'primary', 'school', 'asda', 'supermarket', 'rowley_regis', 'train', 'station', 'junction', 'epc_rating', 'tbc_admin', 'fees_apply']
['dickens_heath', 'village', 'offer', 'contemporary', 'life', 'style', 'superb', 'range', 'family', 'apartment', 'restaurant', 'office', 'shop', 'medical', 'surgery', 'local', 'library', 'village', 'hall', 'village', 'green', 'provide', 'community', 'lifestyle', 'set', 'easy', 'access', 'train', 'station', 'village', 'ideal', 'family', 'commuter', 'property', 'set', 'road', 'overlook', 'pleasan